In [1]:
# Advent of Code, Day 10 - Jim Carson. 
import numpy as np
DEBUG = False
def parse(puzzle_input):
    with open(puzzle_input,"r") as fp:
        f = fp.read().splitlines()
    return f

onoff = {  '(' : ')',  '[' : ']',  '<' : '>',  '{' : '}',
           ')' : '(',  ']' : '[',  '>' : '<',  '}' : '{',
}
corrupt = { ')': 3, ']': 57, '}': 1197, '>': 25137 }
fix = { '(': 1, '[': 2, '{': 3, '<': 4 }

testdata = parse("input_files/day10.test.txt")
data = parse("input_files/day10.txt")

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

    {([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
    [[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
    [{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
    [<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
    <{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.

Stop at the first incorrect closing character on each corrupted line.

In [2]:
def find_corrupted_lines(i, corrupt_only=False, debug=DEBUG):
    score = 0
    v = []
    for s in list(i):
        if s in ["[", "(", "{", "<"]:
            v.append(s)
        elif s in ["]", ")", "}", ">"]:
            q = v.pop()
            if onoff[q] != s:
                if corrupt_only:
                    if debug:
                        print("Expected %s, but found %s instead" % (onoff[q],s))
                    return corrupt[s]
                return 0
    if corrupt_only:
        return 0
    complete_by = []
    for i in v:
        complete_by.insert(0,onoff[i])
    for j in list(complete_by):
        score = score * 5 + fix[onoff[j]]
    if debug:
        print("Complete by adding %s" % "".join(complete_by), score)
    return (score)
    

To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

    ): 3 points.
    ]: 57 points.
    }: 1197 points.
    >: 25137 points.

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [3]:
# Part 1: 166191
np.sum([find_corrupted_lines(i, True) for i in data])

166191

Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

    ): 1 point.
    ]: 2 points.
    }: 3 points.
    >: 4 points.

So, ])}> would be scored as follows:

    Start with a total score of 0.
    Multiply the total score by 5 to get 0, then add the value of ] (2) to get a new total score of 2.
    Multiply the total score by 5 to get 10, then add the value of ) (1) to get a new total score of 11.
    Multiply the total score by 5 to get 55, then add the value of } (3) to get a new total score of 58.
    Multiply the total score by 5 to get 290, then add the value of > (4) to get a new total score of 294.

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the **middle** score. (There will always be an odd number of scores to consider.) 

In [4]:
# Part 2:  1152088313
scorelist = []
for i in data:
    s = find_corrupted_lines(i, False)
    if s > 0:
        scorelist.append(s)
np.median(scorelist)

1152088313.0